# SPY IV Percentile Analysis

This notebook analyzes SPY's Implied Volatility Percentile over the past year (252 trading days).

**IV Percentile** = (# Days with lower IV than today) / (# Trading Days in lookback period) × 100

## Contents:
1. Current IV Percentile Statistics
2. Historical IV Percentile vs SPY Price Chart
3. Key Insights

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import yfinance as yf
import sys
import os

# Add project root to path
sys.path.insert(0, os.path.dirname(os.path.abspath('')))

from src.data_fetchers.synthetic_generator import SyntheticOptionsGenerator

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11

## 1. Fetch and Calculate SPY IV Percentile Data

In [2]:
# Fetch data for the past 1.5 years (to have enough data for 252-day lookback)
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=550)).strftime('%Y-%m-%d')

print(f"Fetching SPY data from {start_date} to {end_date}...")

# Initialize generator
generator = SyntheticOptionsGenerator(
    symbol='SPY',
    risk_free_rate=0.04,
    dividend_yield=0.015,
    volatility_window=30,
    use_vix_for_iv=True
)

# Fetch underlying data with IV Percentile calculation
data = generator.fetch_underlying_data(start_date, end_date)

print(f"\n✓ Data fetched successfully!")
print(f"  Total days: {len(data)}")
print(f"  Date range: {data.index[0].strftime('%Y-%m-%d')} to {data.index[-1].strftime('%Y-%m-%d')}")

Fetching SPY data from 2024-05-27 to 2025-11-28...
Fetching SPY price data from Yahoo Finance...
Date range: 2024-05-27 to 2025-11-28
Fetching VIX data from Yahoo Finance...
✓ VIX data merged (for reference)
  VIX range: 11.94 - 52.33
Calculating SPY implied volatility...
✓ SPY IV Percentile calculated (252-day lookback)
  SPY IV range: 11.94% - 52.33%
  IV Percentile range: 5.6% - 100.0%
✓ Retrieved 378 days of price data
  Price range: $493.65 - $687.39
  Volatility range: 6.69% - 45.46%

✓ Data fetched successfully!
  Total days: 378
  Date range: 2024-05-28 to 2025-11-26


## 2. Current IV Percentile Statistics

In [3]:
# Get data for the past year (252 trading days)
past_year_data = data.iloc[-252:].copy()

# Current (most recent) values
current_date = data.index[-1]
current_spy_price = data['close'].iloc[-1]
current_spy_iv = data['spy_iv'].iloc[-1]
current_iv_percentile = data['iv_percentile'].iloc[-1]

# Statistics for past year
max_iv_percentile = past_year_data['iv_percentile'].max()
min_iv_percentile = past_year_data['iv_percentile'].min()
avg_iv_percentile = past_year_data['iv_percentile'].mean()

max_spy_iv = past_year_data['spy_iv'].max()
min_spy_iv = past_year_data['spy_iv'].min()

# Dates of extremes
max_iv_percentile_date = past_year_data['iv_percentile'].idxmax()
min_iv_percentile_date = past_year_data['iv_percentile'].idxmin()

print("="*70)
print("SPY IMPLIED VOLATILITY PERCENTILE ANALYSIS")
print("="*70)
print(f"\nAs of: {current_date.strftime('%Y-%m-%d')}")
print(f"\nCURRENT VALUES:")
print(f"  SPY Price:           ${current_spy_price:.2f}")
print(f"  SPY Implied Vol:     {current_spy_iv:.2%}")
print(f"  IV Percentile:       {current_iv_percentile:.1f}%")

print(f"\nPAST YEAR STATISTICS (252 Trading Days):")
print(f"  Highest IV Percentile:  {max_iv_percentile:.1f}%  (on {max_iv_percentile_date.strftime('%Y-%m-%d')})")
print(f"  Lowest IV Percentile:   {min_iv_percentile:.1f}%  (on {min_iv_percentile_date.strftime('%Y-%m-%d')})")
print(f"  Average IV Percentile:  {avg_iv_percentile:.1f}%")

print(f"\n  Highest SPY IV:         {max_spy_iv:.2%}")
print(f"  Lowest SPY IV:          {min_spy_iv:.2%}")

print(f"\nINTERPRETATION:")
if current_iv_percentile < 25:
    interpretation = "LOW - Current IV is in the bottom quartile. Consider long options strategies."
elif current_iv_percentile < 50:
    interpretation = "BELOW AVERAGE - Current IV is below median. Moderate environment."
elif current_iv_percentile < 75:
    interpretation = "ABOVE AVERAGE - Current IV is above median. Good for credit spreads."
else:
    interpretation = "HIGH - Current IV is in the top quartile. Excellent for selling premium."

print(f"  {interpretation}")
print("="*70)

SPY IMPLIED VOLATILITY PERCENTILE ANALYSIS

As of: 2025-11-26

CURRENT VALUES:
  SPY Price:           $679.68
  SPY Implied Vol:     17.19%
  IV Percentile:       48.6%

PAST YEAR STATISTICS (252 Trading Days):
  Highest IV Percentile:  100.0%  (on 2025-04-04)
  Lowest IV Percentile:   5.6%  (on 2025-08-22)
  Average IV Percentile:  55.0%

  Highest SPY IV:         52.33%
  Lowest SPY IV:          12.77%

INTERPRETATION:
  BELOW AVERAGE - Current IV is below median. Moderate environment.


## 3. Dual-Axis Chart: SPY Price vs IV Percentile

In [ ]:
# Create figure with dual y-axes
fig, ax1 = plt.subplots(figsize=(16, 9))

# Plot SPY Price on left y-axis
color_spy = '#1f77b4'  # Blue
ax1.set_xlabel('Date', fontsize=12, fontweight='bold')
ax1.set_ylabel('SPY Price ($)', color=color_spy, fontsize=12, fontweight='bold')
ax1.plot(past_year_data.index, past_year_data['close'], 
         color=color_spy, linewidth=2, label='SPY Price')
ax1.fill_between(past_year_data.index, past_year_data['close'], 
                 alpha=0.2, color='lightblue')
ax1.tick_params(axis='y', labelcolor=color_spy)
ax1.grid(True, alpha=0.3)

# Create second y-axis for IV Percentile
ax2 = ax1.twinx()
color_iv = '#d62728'  # Red
ax2.set_ylabel('IV Percentile (%)', color=color_iv, fontsize=12, fontweight='bold')
ax2.plot(past_year_data.index, past_year_data['iv_percentile'], 
         color=color_iv, linewidth=2, label='IV Percentile', linestyle='-')
ax2.tick_params(axis='y', labelcolor=color_iv)
ax2.set_ylim(0, 100)

# Add horizontal reference lines for IV Percentile
ax2.axhline(y=25, color='gray', linestyle='--', alpha=0.5, linewidth=1)
ax2.axhline(y=50, color='gray', linestyle='--', alpha=0.5, linewidth=1)
ax2.axhline(y=75, color='gray', linestyle='--', alpha=0.5, linewidth=1)

# Add text labels for reference lines
ax2.text(past_year_data.index[10], 26, '25th percentile', fontsize=9, color='gray')
ax2.text(past_year_data.index[10], 51, 'Median', fontsize=9, color='gray')
ax2.text(past_year_data.index[10], 76, '75th percentile', fontsize=9, color='gray')

# Format x-axis
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=2))
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45, ha='right')

# Title and legends
plt.title('SPY Price vs IV Percentile (Past Year)', 
          fontsize=16, fontweight='bold', pad=20)

# Create combined legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=11)

# Add current value annotations
ax1.annotate(f'Current: ${current_spy_price:.2f}',
            xy=(current_date, current_spy_price),
            xytext=(10, 10), textcoords='offset points',
            bbox=dict(boxstyle='round,pad=0.5', fc=color_spy, alpha=0.2),
            fontsize=10, fontweight='bold', color=color_spy)

ax2.annotate(f'Current: {current_iv_percentile:.1f}%',
            xy=(current_date, current_iv_percentile),
            xytext=(10, -30), textcoords='offset points',
            bbox=dict(boxstyle='round,pad=0.5', fc=color_iv, alpha=0.2),
            fontsize=10, fontweight='bold', color=color_iv)

plt.tight_layout()
plt.show()

print("\n✓ Chart generated successfully!")

## 4. Additional Analysis: IV Percentile Distribution

In [ ]:
# Create histogram of IV Percentile distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Histogram
ax1.hist(past_year_data['iv_percentile'], bins=20, color='steelblue', 
         alpha=0.7, edgecolor='black')
ax1.axvline(current_iv_percentile, color='red', linestyle='--', 
           linewidth=2, label=f'Current: {current_iv_percentile:.1f}%')
ax1.axvline(avg_iv_percentile, color='green', linestyle='--', 
           linewidth=2, label=f'Average: {avg_iv_percentile:.1f}%')
ax1.set_xlabel('IV Percentile (%)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Number of Days', fontsize=12, fontweight='bold')
ax1.set_title('IV Percentile Distribution (Past Year)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# SPY IV over time
ax2.plot(past_year_data.index, past_year_data['spy_iv'] * 100, 
         color='purple', linewidth=2)
ax2.fill_between(past_year_data.index, past_year_data['spy_iv'] * 100, 
                 alpha=0.2, color='purple')
ax2.set_xlabel('Date', fontsize=12, fontweight='bold')
ax2.set_ylabel('SPY Implied Volatility (%)', fontsize=12, fontweight='bold')
ax2.set_title('SPY Implied Volatility Over Time', fontsize=14, fontweight='bold')
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax2.xaxis.set_major_locator(mdates.MonthLocator(interval=2))
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45, ha='right')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Trading Strategy Recommendations

In [ ]:
print("="*70)
print("TRADING STRATEGY RECOMMENDATIONS")
print("="*70)
print(f"\nBased on Current IV Percentile: {current_iv_percentile:.1f}%\n")

if current_iv_percentile < 25:
    print("🔵 LOW IV ENVIRONMENT (< 25th percentile)")
    print("   Recommended Strategies:")
    print("   • Long options (buying calls/puts)")
    print("   • Debit spreads (Bull Call, Bear Put)")
    print("   • Calendar spreads (buy premium)")
    print("   ")
    print("   Avoid:")
    print("   • Selling naked options (low premium)")
    print("   • Credit spreads (minimal credit received)")
    
elif current_iv_percentile < 50:
    print("🟡 MODERATE IV ENVIRONMENT (25-50th percentile)")
    print("   Recommended Strategies:")
    print("   • Mixed approach")
    print("   • Neutral strategies (Iron Condors, Butterflies)")
    print("   • Small debit spreads")
    
elif current_iv_percentile < 75:
    print("🟠 ELEVATED IV ENVIRONMENT (50-75th percentile)")
    print("   Recommended Strategies:")
    print("   • Credit spreads (Bull Put, Bear Call)")
    print("   • Iron Condors")
    print("   • Covered calls")
    print("   • Cash-secured puts")
    print("   ")
    print("   Avoid:")
    print("   • Long options (expensive premium)")
    
else:
    print("🔴 HIGH IV ENVIRONMENT (> 75th percentile)")
    print("   Recommended Strategies:")
    print("   • Aggressively sell premium")
    print("   • Credit spreads (maximize credit)")
    print("   • Iron Condors (wider wings)")
    print("   • Covered calls (high premium)")
    print("   ")
    print("   ⚠️  Warning: High IV often indicates market stress")
    print("   • Use appropriate position sizing")
    print("   • Consider wider strikes for safety")

print("\n" + "="*70)
print("\nNote: Always consider market conditions, trend, and your risk tolerance")
print("      when selecting strategies. IV Percentile is just one factor.")
print("="*70)

## 6. Export Data (Optional)

In [ ]:
# Uncomment to export data to CSV
# output_file = f'data/processed/spy_iv_percentile_{datetime.now().strftime("%Y%m%d")}.csv'
# past_year_data[['close', 'spy_iv', 'iv_percentile', 'vix']].to_csv(output_file)
# print(f"Data exported to: {output_file}")